### 使用ChatGLM和 Langchain 实现一个客服机器人

客服机器人的场景，包含以下功能：
1. 该机器人能根据用户的输入判断用户要咨询的问题类型，进行自动调用相应的方法，进行返回。
2. 能处理的问题类型有：
    * 预定机票
    * 推荐商品
    * 订单查询
    * 一般购买咨询
3. 要能够实现多轮对话，例如预定机票的场景下，用户提出要订机票，机器人询问要订的日期和航班信息，用户提供信息。如果用户提供的信息不完全，则会再次询问，直到获取了所有的信息，能够完成预定。

我们使用LangChain 的 Agent 和Tool 的功能，实现一个能晚上上述功能的机器人客服。

In [1]:
# 如果没安装，先安装需要的库
# %pip install langchain==0.0.176

In [9]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info("test")

INFO:root:test


首先还是从Sagemaker Endpoint 中生成 Model：

In [10]:
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.chains import RetrievalQA #, RetrievalQAWithSourcesChain
# from langchain.prompts import PromptTemplate

sagemaker_endpoint_name = "mt-chatglm-6b-entpoint"
#sagemaker_endpoint_name = "chinese-alpaca-plus-7b"

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input = {"ask": prompt, **model_kwargs}
        logger.info("prompt: %s", prompt)
        logger.info("model_kwargs: %s", model_kwargs)
        input_str = json.dumps(input)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        logger.info("response_json: %s", response_json)
        return response_json["answer"]

content_handler = ContentHandler()

chatglm_model = SagemakerEndpoint(
    endpoint_name="mt-chatglm-6b-entpoint",
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.001, "top_p": 0.3},
    content_handler=content_handler
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


生成一个 Langchain 的 Agent 和 tools。

使用中文的tool名称和描述信息

In [11]:
from langchain.agents import initialize_agent, Tool

def search_order(input: str) -> str:
    return "订单状态：已发货；发货日期：2023-05-01；预计送达时间：2023-05-10"

def recommend_product(input: str) -> str:
    return "黑色西装外套"

def faq(intput: str) -> str:
    return "7天无理由退货"

def order_flight(flight_date: str, flight_no: str) -> str:
    return "预定：" + flight_date + ", 航班号：" + flight_no

tools_cn = [
    Tool(
        name = "查询订单",func=search_order, 
        description="用于回答客户有关订单的问题"
    ),
    Tool(name="推荐商品", func=recommend_product, 
         description="用于回答客户有关商品推荐的问题"
    ),
    Tool(name="FAQ", func=faq,
         description="用于回答客户一般问题，如发货时间、退换货政策等。"
    ),
    Tool(name="预定航班", func=order_flight,
         description="用户回答客户有关航班预定的问题"
    )
]
agent_cn = initialize_agent(tools_cn, chatglm_model, agent="zero-shot-react-description", verbose=True)

In [9]:
question = "我想买个衬衣，可以给我推荐合适的尺码吗？"
result = agent_cn.run(question)
print(result)

# 还是没有按照提示词思考，直接尝试回答了，结果就出错了



> Entering new AgentExecutor chain...
I'd be happy to help with your衬衫 recommendation! What size do you need?
Action: Please provide me with your desired衬衫 size.
Action Input: Size of衬衫 (e.g. 32, 36, 40, etc.)
Observation: Please provide me with your desired衬衫 size. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I'm sorry, but I cannot provide you with a recommendation for a specific衬衫 size based on the information you provided. To determine the appropriate尺码 for a particular衬衫， you should refer to the manufacturer's specifications or check the recommended尺码 range on the衬衫's label. If you have any further questions or need assistance with finding the right衬衫， please feel free to ask.`

可以看到，使用 ChatGLM 无法准确的按照Prompt 进行思考，然后按照需要的格式生成结果，而是尝试回答了问题，有时还会编造一个答案（实际上就是按模型回答了）。

In [7]:
# 再问一个别的问题：
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent_cn.run(question)
print(result)



> Entering new AgentExecutor chain...
I'm sorry to hear that your order hasn't been delivered. Can you please provide me with the订单号 2022ABCDE so that I can check the status of your order?


Action: Please enter the订单号 2022ABCDE into the input field.

Action Input: 2022ABCDE

Observation: Please enter the订单号 2022ABCDE into the input field. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I'm sorry, but it seems that the input "2022ABCDE" is not a valid订单号. Can you please provide me with a valid订单号 so that I can check the status of your order?`

### 测试结果

不管使用中文、英文的工具描述，同样不能正确的生成结果。这是由于 ChatGLM不能按照提示词进行“思考”，不能按要求的格式返回。

而且，返回的结果，中文英文混杂，这一点可以通过自定义prompt改善。

至于让模型按LangChain的要求进行“思考”，需要在LangChain提供的这个Agent的默认提示词里添加控制指令。但是使用ChatGLM始终无法达到预期效果。

#### 使用 ChatGPT 执行

下面使用 ChatGPT，看看它的思考过程是怎么样的。
使用之前的tools工具链，并用OpenAI的ChatGPT创建 Agent。需要先安装依赖。

In [10]:
# %pip install openai

In [41]:
import os
from langchain.llms import OpenAI
os.environ["OPENAI_API_KEY"] = "sk-xxx"
llm_openai = OpenAI(temperature=0.01, top_p=0.75)
agent_eng_openai = initialize_agent(tools, llm_openai, agent="zero-shot-react-description", verbose=True)

In [42]:
agent_eng_openai.run("你们的退货政策是怎么样的?")



> Entering new AgentExecutor chain...
 I need to find out the return policy
Action: FAQ
Action Input: return policy
Observation: 7天无理由退货
Thought: I now know the final answer
Final Answer: 我们的退货政策是7天无理由退货。

> Finished chain.


'我们的退货政策是7天无理由退货。'

In [43]:
agent_eng_openai("我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？")



> Entering new AgentExecutor chain...
 I need to recommend a product to the user.
Action: Recommend Product
Action Input: "dress"
Observation: 红色连衣裙
Thought: I now know the final answer.
Final Answer: 我推荐红色连衣裙，它看起来很漂亮。

> Finished chain.


{'input': '我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'output': '我推荐红色连衣裙，它看起来很漂亮。'}

可以从详情日志中看到 ChatGPT “思考”的过程。他会按 “Action”，“Action Input”，“Observation”，“Thought” 和 “Final Answer”生成结果，其中 “Observation” 是函数执行的结果。最后，LangChain框架会解析这个结果，产生最终结果。

再尝试有两个参数的。

In [44]:
agent_eng_openai.run("帮我预定明晚到上海的飞机")



> Entering new AgentExecutor chain...
 I need to help the customer book a flight
Action: order flight
Action Input: flight to Shanghai tomorrow night

TypeError: <lambda>() missing 1 required positional argument: 'flight_no'

执行出错是因为 LangChain 的Agent所执行的Tool，只支持一个参数，我们的航班预订的 “order_flight” 方法有两个参数，而模型只识别出一个参数并尝试调用函数，结果就出错了。

如果工具需要多个参数，需要使用其他方式创建工具。

> 新版的 LangChain 已经支持多个参数，但是需要通过 *StructuredTool* 封装tool。

#### 自定义实现
下面使用自定义的方式实现。实现思路：
1. 使用embeddings向量搜索，根据用户输入的问题，找到最匹配的方法
2. 识别用户输入中该方法所需要的参数，例如航班订阅，需要日期和航班号
3. 调用该python方法。

In [15]:
# 安装需要的依赖库
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 79.5 MB/s eta 0:00:0000:0100:01


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='GanymedeNil/text2vec-large-chinese')

No sentence-transformers model found with name /home/ec2-user/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


创建函数，并将函数名称进行 Embedding 保存。

In [9]:
import re
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

ORDER_1 = "20230101ABC"
ORDER_2 = "20230101EFG"
ORDER_1_DETAIL = { "order_number": ORDER_1, "status": "已发货", "shipping_date" : "2023-01-03", "estimated_delivered_date": "2023-01-05",} 
ORDER_2_DETAIL = { "order_number": ORDER_2, "status": "未发货", "shipping_date" : None, "estimated_delivered_date": None,}

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
answer_order_info = PromptTemplate(
    template="请把下面的订单信息回复给用户： \n\n {order}?", input_variables=["order"]
)
answer_order_llm = LLMChain(llm=chatglm_model,  prompt=answer_order_info)

from langchain.agents import initialize_agent, Tool, tool

def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
    return "7天无理由退货"

def recommend_product(input: str) -> str:
    """"useful for when you need to search and recommend products and recommend it to the user"""
    return "红色连衣裙"

def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    pattern = r"\d+[A-Z]+"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "请问您的订单号是多少？"
    if order_number == ORDER_1:        
        return answer_order_llm.run(json.dumps(ORDER_1_DETAIL))
    elif order_number == ORDER_2:
        return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
    else:
        return f"对不起，根据{input}没有找到您的订单"

    
def order_flight(flightDate: str, flightNo: str):
    if not flightDate and not flightNo:
        return "请提供出行日期和预定航班"
    elif not flightDate:
        return "请提供出行日期"
    elif not flightNo:
        return "请提供航班号"
    
    return "预定 " + flightDate + " 的航班，航班号是：" + flightNo

funcs = {"查询商品购买订单": search_order, "商品推荐": recommend_product, "问答-退换货及商品购买咨询": faq, "预定航班": order_flight}
func_inputs = {"查询商品购买订单": ['订单号'], "商品推荐": ['喜欢的风格'], "问答-退换货及商品购买咨询": ['text'], "预定航班": ['出行日期', '航班号']}
func_input_map = {"出行日期": "flightDate", "航班号": "flightNo", "喜欢的风格": "input", "订单号": "input"}
func_doc = [Document(page_content=t, metadata={}) for t in funcs]
func_index = FAISS.from_documents(func_doc, embeddings)

当用户提问时，使用这个问题进行详细搜索，就能得到所需要调用的函数，达到意图识别的目的。

In [10]:
func_index.similarity_search("Human: 我的订单一直没收到货\nAI: 请提供订单号\ninput: 20230101EFG", k=1)

[Document(page_content='查询商品购买订单', metadata={})]

下面使用 ChatGLM 创建 LangChain 的 Chain，我们会用这个，从用户的提问中提取所需要调用的函数的参数。

我希望用json返回提取的结果，如提取订单号，那就返回 {"订单号": "2023ABC"} 这样的值。

In [14]:
from langchain import PromptTemplate, LLMChain

prompt_template = """
从下面的用户输入中提取信息, 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式返回:
用户: {question}
提取: [{input}]
"""
prompt = PromptTemplate(
    input_variables=["question", "input"],
    template=prompt_template
)

llm_chain = LLMChain(llm=chatglm_model, prompt=prompt)

In [15]:
import json

def call_func(the_func, inputs):
    params = {}
    for pkey in inputs:
        params[func_input_map[pkey]] = inputs.get(pkey)
    print('the params:', params)
    result = the_func(**params)
    print('the func result:', result)

def chat_bot(query):
    result = func_index.similarity_search(query, k=1)
    the_fun = funcs[result[0].page_content]
    the_fun_inputs = func_inputs[result[0].page_content]
    the_input = ",".join(the_fun_inputs)
    
    if the_input == 'text':
        return the_fun(the_input)
    else:
        prompt = prompt_template.replace("{question}", query).replace("{input}", the_input)
        print("promt:", prompt)

        result = llm_chain({"question": query, "input": the_input})
        print('提取结果:', result)
        if result['text'].find('很抱歉') >= 0:
            return '很抱歉未找到所需的信息，请提供 ' + the_input
        result_text = result['text'].replace('\n', '')
        print('提取结果json:', result_text)
        input_obj = json.loads(result_text)
        call_func(the_fun, input_obj)
        return result


In [19]:
chat_bot("我的订单没收到")

promt: 
从下面的用户输入中提取信息, 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式返回:
用户: 我的订单没收到
提取: [订单号]

提取结果: {'question': '我的订单没收到', 'input': '订单号', 'text': '无法完成这个任务，因为要求从用户输入中提取信息，并且要求使用已有知识，而不是尝试回答问题或编造答案。此外，要求返回json格式的结果，但并没有提供任何有关订单号的信息。请提供更多上下文或详细信息，以便我可以更好地理解并提供帮助。'}
提取结果json: 无法完成这个任务，因为要求从用户输入中提取信息，并且要求使用已有知识，而不是尝试回答问题或编造答案。此外，要求返回json格式的结果，但并没有提供任何有关订单号的信息。请提供更多上下文或详细信息，以便我可以更好地理解并提供帮助。


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
# 因为会使用多轮对话，所以模拟多轮对话，即先是问题，再是订单号的方式
chat_bot("我的订单没收到, 订单号 20230101EFG")

promt: 
从下面的用户输入中提取信息, 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式返回:
用户: 我的订单没收到
 请提供订单号
 20230101EFG
提取: [订单号]

提取结果: {'question': '我的订单没收到\n 请提供订单号\n 20230101EFG', 'input': '订单号', 'text': '{\n "订单号": "20230101EFG"\n}'}
提取结果json: { "订单号": "20230101EFG"}
the params: {'input': '20230101EFG'}
the func result: 很抱歉，您提供的信息似乎不完整，我无法为您提供完整的订单信息。请提供更多上下文或完整的订单信息，以便我可以更好地回答您的问题。


{'question': '我的订单没收到\n 请提供订单号\n 20230101EFG',
 'input': '订单号',
 'text': '{\n "订单号": "20230101EFG"\n}'}

In [17]:
chat_bot("我购买的商品没有收到")

promt: 
从下面的用户输入中提取信息, 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式返回:
用户: 我购买的商品没有收到
提取: [订单号]

提取结果: {'question': '我购买的商品没有收到', 'input': '订单号', 'text': '很抱歉，根据您的要求，我无法准确从用户输入中提取信息。如果您能提供更多信息，我将尽力帮助您。'}


'很抱歉未找到所需的信息，请提供 订单号'

In [18]:
chat_bot("想预定这个月8号到北京的机票")

promt: 
从下面的用户输入中提取信息, 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式返回:
用户: 想预定这个月8号到北京的机票
提取: [出行日期,航班号]

提取结果: {'question': '想预定这个月8号到北京的机票', 'input': '出行日期,航班号', 'text': '{  \n "出行日期": "这个月8号",  \n "航班号": "北京到地球的航班"  \n}'}
提取结果json: {   "出行日期": "这个月8号",   "航班号": "北京到地球的航班"  }
the params: {'flightDate': '这个月8号', 'flightNo': '北京到地球的航班'}
the func result: 预定 这个月8号 的航班，航班号是：北京到地球的航班


{'question': '想预定这个月8号到北京的机票',
 'input': '出行日期,航班号',
 'text': '{  \n "出行日期": "这个月8号",  \n "航班号": "北京到地球的航班"  \n}'}

### 测试结果

有时候ChatGLM无法准确的按照要求返回结果，有时候结果不是准确的json格式，当无法提取信息的时候，会返回text，这时候需要先判断一下回答的内容是否有“无法完成这个任务”或“抱歉”这样的词。

#### 尝试使用 Chinese-LLaMA-Alpaca

In [20]:
sagemaker_endpoint_name = "chinese-alpaca-plus-7b"

alpaca_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.001, "top_p": 0.3},
    content_handler=content_handler
)

In [21]:
from langchain import PromptTemplate, LLMChain

prompt_template = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction: 从以下对话中信息提取'{input}', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.\n"
    "### Input: {question} \n\n"
    "### Response: "
)
prompt = PromptTemplate(
    input_variables=["question", "input"],
    template=prompt_template
)

llm_chain = LLMChain(llm=alpaca_model, prompt=prompt)

In [22]:
chat_bot("我的订单没收到,订单号是 20230101EFG")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对话中信息提取'订单号', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
### Input: 我的订单没收到,订单号是 20230101EFG 

### Response: 
提取结果: {'question': '我的订单没收到,订单号是 20230101EFG', 'input': '订单号', 'text': '{ "order_number": "20230101EFG" }'}
提取结果json: { "order_number": "20230101EFG" }


KeyError: 'order_number'

生成的结果中的key都是英文，尝试了几种提示词，都是英文。所以修改原先的metadata，使用英文的参数key。

In [23]:
func_inputs = {"查询商品购买订单": ['order_number'], "商品推荐": ['favor_style'], "问答-退换货及商品购买咨询": ['text'], "预定航班": ['flight_date', 'flight_no']}
func_input_map = {"flight_date": "flightDate", "flight_no": "flightNo", "favor_style": "input", "order_number": "input"}

In [24]:
chat_bot("我的订单没收到,订单号是 20230101EFG")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对话中信息提取'order_number', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
### Input: 我的订单没收到,订单号是 20230101EFG 

### Response: 
提取结果: {'question': '我的订单没收到,订单号是 20230101EFG', 'input': 'order_number', 'text': '{ "order_number": "20230101EFG" }'}
提取结果json: { "order_number": "20230101EFG" }
the params: {'input': '20230101EFG'}
the func result: 很抱歉，您提供的信息似乎不完整，我无法为您提供完整的订单信息。请提供更多上下文或完整的订单信息，以便我可以更好地回答您的问题。


{'question': '我的订单没收到,订单号是 20230101EFG',
 'input': 'order_number',
 'text': '{ "order_number": "20230101EFG" }'}

In [25]:
chat_bot("帮我预定明晚到上海的航班")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对话中信息提取'flight_date,flight_no', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
### Input: 帮我预定明晚到上海的航班 

### Response: 
提取结果: {'question': '帮我预定明晚到上海的航班', 'input': 'flight_date,flight_no', 'text': '{ "flight_date": "明晚", "flight_no": "" }'}
提取结果json: { "flight_date": "明晚", "flight_no": "" }
the params: {'flightDate': '明晚', 'flightNo': ''}
the func result: 请提供航班号


{'question': '帮我预定明晚到上海的航班',
 'input': 'flight_date,flight_no',
 'text': '{ "flight_date": "明晚", "flight_no": "" }'}

In [26]:
chat_bot("我要订机票，22号的，航班CA1478")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对话中信息提取'flight_date,flight_no', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
### Input: 我要订机票，22号的，航班CA1478 

### Response: 
提取结果: {'question': '我要订机票，22号的，航班CA1478', 'input': 'flight_date,flight_no', 'text': '{ "flight_date": "22", "flight_no": "CA1478" }'}
提取结果json: { "flight_date": "22", "flight_no": "CA1478" }
the params: {'flightDate': '22', 'flightNo': 'CA1478'}
the func result: 预定 22 的航班，航班号是：CA1478


{'question': '我要订机票，22号的，航班CA1478',
 'input': 'flight_date,flight_no',
 'text': '{ "flight_date": "22", "flight_no": "CA1478" }'}

In [27]:
chat_bot("我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对话中信息提取'favor_style', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
### Input: 我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？ 

### Response: 
提取结果: {'question': '我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'input': 'favor_style', 'text': '{ "favor_style": "裙子" }'}
提取结果json: { "favor_style": "裙子" }
the params: {'input': '裙子'}
the func result: 红色连衣裙


{'question': '我想买一条裙子，但是不知道哪个款式好看，你能帮我推荐一下吗？',
 'input': 'favor_style',
 'text': '{ "favor_style": "裙子" }'}

In [92]:
chat_bot("我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗?")

promt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
从以下对话中信息提取'order_number', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 结果用json格式,使用原先的key.
我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗?
### Response: 
提取结果: {'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗?', 'input': 'order_number', 'text': '{ "order_number": "2022ABCDE" }'}
提取结果json: { "order_number": "2022ABCDE" }
the params: {'input': '2022ABCDE'}
the func result: 对不起，根据2022ABCDE没有找到您的订单


{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗?',
 'input': 'order_number',
 'text': '{ "order_number": "2022ABCDE" }'}

可以看到，这几个都能够正常的识别，并且正确的调用相应的function，得到预期的结果。